In [ ]:
!uv pip install lightgbm optuna scikit-learn pandas matplotlib seaborn IProgress jupyter ipywidgets -U -q

In [ ]:
!uv pip install ../../target/wheels/perpetual-1.0.0-cp313-cp313-win_amd64.whl

In [ ]:
import math
import optuna
import numpy as np
import pandas as pd
from functools import partial
from time import process_time, time
from lightgbm import LGBMClassifier
from sklearn.utils import resample
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_validate
from perpetual import PerpetualBooster

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
from sklearn.datasets import fetch_openml


data = fetch_openml(name='TVS_Loan_Default', version=1, return_X_y=False, as_frame=True)
X = data.frame

In [ ]:
df_majority = X[X['V32'] == 0]
df_minority = X[X['V32'] != 0]

print(f"Majority (zeros) size: {len(df_majority)}")
print(f"Minority (non-zeros) size: {len(df_minority)}")

n_samples = len(df_minority) * 10

df_majority_undersampled = resample(
    df_majority,
    replace=False,        # Sample without replacement
    n_samples=n_samples,  # Match number of samples in minority class
    random_state=42       # Set a seed for reproducibility
)

print(f"Undersampled majority size: {len(df_majority_undersampled)}")

X = pd.concat([df_minority, df_majority_undersampled])

print("\nClass distribution in the balanced dataframe:")
print(X['V32'].value_counts())

In [ ]:
y = X.pop('V32')

In [ ]:
object_cols = X.select_dtypes(include=['object']).columns
X[object_cols] = X[object_cols].astype('category')

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
np.mean(y.values)

In [ ]:
y.value_counts()

In [ ]:
X.head()

In [ ]:
X.drop(columns=["V16"], inplace=True)

In [ ]:
def prepare_data(seed):
    scoring = "neg_log_loss"
    metric_function = log_loss
    metric_name = "log_loss"
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=seed)

    return (
        X_train,
        X_test,
        y_train,
        y_test,
        scoring,
        metric_function,
        metric_name,
    )


def objective_function(trial, seed, n_estimators, X_train, y_train, scoring):
    
    params = {
        "seed": seed,
        "verbosity": -1,
        "n_estimators": n_estimators,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.5, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 1e-6, 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-6, 1.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-6, 1.0, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "max_depth": trial.suggest_int("max_depth", 3, 33),
        "num_leaves": trial.suggest_int("num_leaves", 2, 1024),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 100),
    }

    model = LGBMClassifier(**params)

    cv_results = cross_validate(
        model,
        X_train,
        y_train,
        cv=5,
        scoring=scoring,
        return_train_score=True,
        return_estimator=True,
    )

    return -1 * np.mean(cv_results["test_score"])

In [ ]:
n_estimators = 100
n_trials = 100
seed = 0

(
    X_train,
    X_test,
    y_train,
    y_test,
    scoring,
    metric_function,
    metric_name,
) = prepare_data(seed)

sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="minimize", sampler=sampler)

obj = partial(
    objective_function,
    seed=seed,
    n_estimators=n_estimators,
    X_train=X_train,
    y_train=y_train,
    scoring=scoring,
)

start = process_time()
tick = time()
study.optimize(obj, n_trials=n_trials)
stop = process_time()


print(f"seed: {seed}, cpu time: {stop - start}")

In [ ]:
study.best_trial.params

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from perpetual.sklearn import PerpetualClassifier
from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay


params = study.best_trial.params
params["n_estimators"] = n_estimators
params["seed"] = seed
params["verbosity"] = -1
perp = PerpetualClassifier(budget=1.5)
lgbm = LGBMClassifier(**params)
lgbm_isotonic = CalibratedClassifierCV(LGBMClassifier(**params), cv=5, method="isotonic")
lgbm_sigmoid = CalibratedClassifierCV(LGBMClassifier(**params), cv=5, method="sigmoid")


clf_list = [
    (perp, "Perpetual"),
    (lgbm, "LightGBM"),
    (lgbm_isotonic, "LightGBM + Isotonic"),
    (lgbm_sigmoid, "LightGBM + Sigmoid"),
]

In [ ]:
n_bins = 10

In [ ]:
fig = plt.figure(figsize=(10, 10))
gs = GridSpec(4, 2)
colors = plt.get_cmap("Dark2")

ax_calibration_curve = fig.add_subplot(gs[:2, :2])
calibration_displays = {}
for i, (clf, name) in enumerate(clf_list):
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)[:, 1]
    display = CalibrationDisplay.from_predictions(
        y_test,
        y_pred,
        n_bins=n_bins,
        name=name,
        ax=ax_calibration_curve,
        color=colors(i),
    )
    calibration_displays[name] = display

ax_calibration_curve.grid()
ax_calibration_curve.set_title("Calibration plots (Naive Bayes)")

# Add histogram
grid_positions = [(2, 0), (2, 1), (3, 0), (3, 1)]
for i, (_, name) in enumerate(clf_list):
    row, col = grid_positions[i]
    ax = fig.add_subplot(gs[row, col])

    ax.hist(
        calibration_displays[name].y_prob,
        range=(0, 1),
        bins=n_bins,
        label=name,
        color=colors(i),
    )
    ax.set(title=name, xlabel="Mean predicted probability", ylabel="Count")

plt.tight_layout()
plt.show()

In [ ]:
perp.classes_

In [ ]:
from lightgbm import plot_importance

plot_importance(clf_list[1][0])

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


y_pred = clf_list[0][0].predict(X_test)
y_proba = clf_list[0][0].predict_proba(X_test)

print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(roc_auc_score(y_test, y_proba[:, 1]))

In [ ]:
import numpy as np
from typing import Union, Tuple, Sequence
from sklearn.calibration import calibration_curve


def expected_calibration_error(y_true: Union[np.ndarray, Sequence[int]], 
                               y_pred: Union[np.ndarray, Sequence[float]], 
                               n_bins: int = 10) -> Tuple[float, np.ndarray, np.ndarray]:
    """
    Calculates the Expected Calibration Error (ECE) for predicted probabilities.

    ECE is the weighted average of the absolute difference between the mean 
    true outcome and the mean predicted probability in each confidence bin. 
    The weights are the proportion of samples falling into each bin.

    Args:
        y_true: True binary labels (0 or 1).
        y_pred: Predicted probabilities for the positive class (between 0 and 1).
        n_bins: The number of bins to use for the calibration curve (default is 10).

    Returns:
        A tuple containing:
        - The calculated ECE (float).
        - The array of mean true probabilities per bin (prob_true).
        - The array of mean predicted probabilities per bin (prob_pred).
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    N = len(y_true)

    # 1. Calculate the mean true probability and mean predicted probability for non-empty bins
    # prob_true and prob_pred only contain values for non-empty bins.
    prob_true, prob_pred = calibration_curve(y_true, y_pred, n_bins=n_bins)

    # 2. Determine the counts for the weights
    # The bins are uniform over [0, 1].
    bins = np.linspace(0., 1., n_bins + 1)
    
    # Calculate the count of samples that fall into each of the n_bins.
    # The range (0, 1) is used implicitly by calibration_curve.
    counts, _ = np.histogram(y_pred, bins=bins, range=(0., 1.))

    # 3. Filter counts to match the non-empty bins returned by calibration_curve.
    # The only bins that contribute to ECE are those that are not empty (count > 0).
    non_empty_counts = counts[counts > 0]
    
    # 4. Calculate the weights (Ni / N)
    # The weight is the fraction of total samples in each non-empty bin.
    weights = non_empty_counts / N

    # 5. Calculate the ECE
    # ECE = sum_i (Weight_i * |prob_true_i - prob_pred_i|)
    ece = np.sum(weights * np.abs(prob_true - prob_pred))

    return ece, prob_true, prob_pred, weights


In [ ]:
print(expected_calibration_error(y_test, clf_list[0][0].predict_proba(X_test)[:,1], n_bins))
print(expected_calibration_error(y_test, clf_list[1][0].predict_proba(X_test)[:,1], n_bins))
print(expected_calibration_error(y_test, clf_list[2][0].predict_proba(X_test)[:,1], n_bins))
print(expected_calibration_error(y_test, clf_list[3][0].predict_proba(X_test)[:,1], n_bins))

In [ ]:
y_pred_train_perp = perp.predict(X_train)
y_proba_train_perp = perp.predict_proba(X_train)

In [ ]:
y_pred_test_perp = perp.predict(X_test)
y_proba_test_perp = perp.predict_proba(X_test)

print(accuracy_score(y_test, y_pred_test_perp))
print(f1_score(y_test, y_pred_test_perp))
print(roc_auc_score(y_test, y_proba_test_perp[:, 1]))

In [ ]:
print(expected_calibration_error(y_test, y_proba_test_perp[:,1], n_bins))

In [ ]:
pred_nodes_test = perp.predict_nodes(X_test)

In [ ]:
def get_leaf_nodes(perp: PerpetualBooster):
    return [{node.num: node for node in tree_nodes if node.is_leaf} for tree_nodes in perp.get_node_lists()]

In [ ]:
def get_weights(leaf_nodes, pred_nodes):
  pred_weights = np.array([
      [
        [
              leaf_nodes[i][key].weights 
              for key in leaf_nodes[i].keys() & set(nodes)
        ][0]
          for nodes in tree_nodes
      ] 
      for i, tree_nodes in enumerate(pred_nodes)
  ])
  
  return np.sort(pred_weights, axis=2)

In [ ]:
bin_edges = np.linspace(0., 1., n_bins + 1)
print(bin_edges)

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

perp_models = []

for i, (train, test) in enumerate(cv.split(X_train, y_train)):
    print(f"Fold {i}")
    X_train_cv, X_test_cv = X_train.iloc[train], X_train.iloc[test]
    y_train_cv, y_test_cv = y_train.iloc[train], y_train.iloc[test]
    perp = PerpetualBooster(budget=1.0)
    perp.fit(X_train_cv, y_train_cv)
    

    pred_weights = get_weights(get_leaf_nodes(perp), perp.predict_nodes(X_test_cv))
    y_proba_cv = perp.predict_proba(X_test_cv)

    bin_indices_cv = np.digitize(y_proba_cv[:,1], bin_edges[1:-1])

    cal_dicts = []

    for j in range(n_bins):
        bin_edge_left = bin_edges[j]
        bin_edge_right = bin_edges[j+1]

        proba_true_test_bin = y_test_cv[bin_indices_cv==j]
        count = len(proba_true_test_bin)
        proba_true_test = np.mean(proba_true_test_bin)
        proba_pred_test = np.mean(y_proba_cv[:,1][bin_indices_cv==j])
        for w_i in range(4):
            weights_lower = np.sum(pred_weights[:, :, w_i], axis=0) + perp.base_score
            weights_upper = np.sum(pred_weights[:, :, w_i+1], axis=0) + perp.base_score
            for k in range(11):
                cal_weight = k / 10
                w = weights_lower * (1 - cal_weight) + weights_upper * cal_weight
                p = 1.0 / (1.0 + np.exp(-w))
                p_bin = p[bin_indices_cv==j]
                p_cal = np.mean(p_bin)
                print(f"Bin: {j}, count: {count}, w_i: {w_i}, cal_weight: {cal_weight:.1f}, proba_true: {proba_true_test}, proba_pred: {proba_pred_test}, p_cal: {p_cal}")
                
                if p_cal > proba_true_test:
                    c_weight = np.interp(proba_true_test, [p_cal_prev, p_cal], [cal_weight_prev, cal_weight])
                    cal_dicts.append({"bin_edges": (bin_edge_left, bin_edge_right), "weight_indices": (w_i, w_i+1), "cal_weight": c_weight})
                    break
                else:
                    p_cal_prev = p_cal
                    cal_weight_prev = cal_weight

            else:
                continue  # only executed if the inner loop did NOT break
            break  # only executed if the inner loop DID break

        if count == 0:
            print(f"Warning: Bin {j} is empty.")
            cal_dicts.append({"bin_edges": (bin_edge_left, bin_edge_right), "weight_indices": (2, 2), "cal_weight": 1.0})
        elif w_i == 3 and k == 10:
            print(f"Warning: Could not calibrate bin {j} with proba_true_test {proba_true_test}")
            cal_dicts.append({"bin_edges": (bin_edge_left, bin_edge_right), "weight_indices": (w_i+1, w_i+1), "cal_weight": 1.0})

    print(cal_dicts)

    perp_models.append((perp, cal_dicts))

    ece, prob_true, prob_pred, weights = expected_calibration_error(y_test_cv, y_proba_cv[:,1], n_bins)

    print(f"ECE: {ece}")
    

In [ ]:
for m, c_d in perp_models:
    print(c_d)

In [ ]:
c_dicts = [dicts for _, dicts in perp_models]
weight_indices_left = [[e["weight_indices"][0] for e in d] for d in c_dicts]
print(weight_indices_left)
weight_indices_left = np.median(np.array(weight_indices_left), axis=0).astype(int)
print(weight_indices_left)
weight_indices_right = [[e["weight_indices"][1] for e in d] for d in c_dicts]
print(weight_indices_right)
weight_indices_right = np.median(np.array(weight_indices_right), axis=0).astype(int)
print(weight_indices_right)
cal_weights = [[e["cal_weight"] for e in d] for d in c_dicts]
print(cal_weights)
cal_weights_median = np.median(np.array(cal_weights), axis=0)
print(cal_weights_median)
cal_weights_mean = np.mean(np.array(cal_weights), axis=0)
print(cal_weights_mean)

In [ ]:
[[e["weight_indices"][0] for e in d] for d in c_dicts]

In [ ]:
[[e["weight_indices"][1] for e in d] for d in c_dicts]

In [ ]:
print(weight_indices_left)
print(weight_indices_right)

In [ ]:
weight_indices_left[2] = 3
weight_indices_right[2] = 4
cal_weights_median[2] = 0.4
weight_indices_left[3] = 4
weight_indices_right[3] = 4
weight_indices_left[4] = 4
weight_indices_right[4] = 4
weight_indices_left[5] = 2
weight_indices_right[5] = 3
cal_weights_median[5] = 0.1
weight_indices_left[6] = 0
weight_indices_right[6] = 1
weight_indices_left[7] = 1
weight_indices_right[7] = 1
weight_indices_left[8] = 2
weight_indices_right[8] = 2

In [ ]:
y_proba = []
y_proba_cal = []

for m, _ in perp_models:
    y_pred = m.predict(X_test)
    y_proba_model = m.predict_proba(X_test)[:,1]
    y_proba.append(y_proba_model)
    pred_weights = get_weights(get_leaf_nodes(m), m.predict_nodes(X_test))
    bin_indices = np.digitize(y_proba_model, bin_edges[1:-1])

    y_proba_cal_model = []
    
    for i, b_i in enumerate(bin_indices):
        #bin_cal_dict = next((item for item in c_dict if item["bin_edges"][0] <= y_proba_model[i] < item["bin_edges"][1]), None)
        #w_i_lower, w_i_upper = bin_cal_dict["weight_indices"]
        #cal_weight = bin_cal_dict["cal_weight"]

        w_i_lower, w_i_upper = weight_indices_left[b_i], weight_indices_right[b_i]
        cal_weight = cal_weights_median[b_i]

        weights_lower = np.sum(pred_weights[:, i, w_i_lower], axis=0) + m.base_score
        weights_upper = np.sum(pred_weights[:, i, w_i_upper], axis=0) + m.base_score

        w = weights_lower * (1 - cal_weight) + weights_upper * cal_weight
        p_cal = 1.0 / (1.0 + np.exp(-w))
        y_proba_cal_model.append(p_cal)

    y_proba_cal.append(y_proba_cal_model)

y_proba = np.mean(np.array(y_proba), axis=0)
y_proba_cal = np.mean(np.array(y_proba_cal), axis=0)

print(expected_calibration_error(y_test, y_proba, n_bins))
print(expected_calibration_error(y_test, y_proba_cal, n_bins))

In [ ]:
(0.03398951-0.03106936)*7.70118940e-01

In [ ]:
figure, axis = plt.subplots(figsize=(8, 8))
disp = CalibrationDisplay.from_predictions(
    y_test,
    y_proba,
    n_bins=n_bins,
    name="perp",
    ax=axis,
    ref_line=True,
)
disp_cal = CalibrationDisplay.from_predictions(
    y_test,
    y_proba_cal,
    n_bins=n_bins,
    name="perp_cal",
    ax=axis,
    ref_line=True,
)
plt.grid(True)
plt.show()

In [ ]:
def objective_cal(trial, models, X_train, y_train, cv):
    y_proba = []
    y_proba_cal = []
    y_train_shuffled = []

    weight_indices_cal = [trial.suggest_float(f"w_i_cal_{i}", 1.0, 4.0) for i in range(n_bins)]

    for i, (train, test) in enumerate(cv.split(X_train, y_train)):

        X_train_cv, X_test_cv = X_train.iloc[train], X_train.iloc[test]
        y_train_cv, y_test_cv = y_train.iloc[train], y_train.iloc[test]

        y_train_shuffled.extend(list(y_test_cv))

        m = models[i]

        y_proba_model = m.predict_proba(X_test_cv)[:,1]
        y_proba.extend(list(y_proba_model))
        pred_weights = get_weights(get_leaf_nodes(m), m.predict_nodes(X_test_cv))
        bin_indices = np.digitize(y_proba_model, bin_edges[1:-1])

        y_proba_cal_model = []
        
        for i, b_i in enumerate(bin_indices):

            cal_weight, w_i_lower = math.modf(weight_indices_cal[b_i])
            w_i_lower = int(w_i_lower)
            w_i_upper = w_i_lower + 1

            weights_lower = np.sum(pred_weights[:, i, w_i_lower], axis=0) + m.base_score
            weights_upper = np.sum(pred_weights[:, i, w_i_upper], axis=0) + m.base_score

            w = weights_lower * (1 - cal_weight) + weights_upper * cal_weight
            p_cal = 1.0 / (1.0 + np.exp(-w))
            y_proba_cal_model.append(float(p_cal))

        y_proba_cal.extend(list(y_proba_cal_model))

    return expected_calibration_error(y_train_shuffled, y_proba_cal, n_bins)[0]




In [ ]:
obj_cal = partial(
    objective_cal,
    models=[m for m, _ in perp_models],
    X_train=X_train,
    y_train=y_train,
    cv=cv,
)
sampler_cal = optuna.samplers.TPESampler(seed=seed)
study_cal = optuna.create_study(direction="minimize", sampler=sampler_cal)
study_cal.optimize(obj_cal, n_trials=100)


In [ ]:
y_proba = []
y_proba_cal = []

weight_indices_cal = [v for k, v in study_cal.best_trial.params.items()]
print(weight_indices_cal)

for m, _ in perp_models:
    y_pred = m.predict(X_test)
    y_proba_model = m.predict_proba(X_test)[:,1]
    y_proba.append(y_proba_model)
    pred_weights = get_weights(get_leaf_nodes(m), m.predict_nodes(X_test))
    bin_indices = np.digitize(y_proba_model, bin_edges[1:-1])

    y_proba_cal_model = []
    
    for i, b_i in enumerate(bin_indices):

        cal_weight, w_i_lower = math.modf(weight_indices_cal[b_i])
        w_i_lower = int(w_i_lower)
        w_i_upper = w_i_lower + 1

        weights_lower = np.sum(pred_weights[:, i, w_i_lower], axis=0) + m.base_score
        weights_upper = np.sum(pred_weights[:, i, w_i_upper], axis=0) + m.base_score

        w = weights_lower * (1 - cal_weight) + weights_upper * cal_weight
        
        y_proba_cal_model.append(w)

    y_proba_cal.append(y_proba_cal_model)

y_proba = np.mean(np.array(y_proba), axis=0)
y_proba_cal = np.mean(np.array(y_proba_cal), axis=0)
y_proba_cal = 1.0 / (1.0 + np.exp(-y_proba_cal))

print(expected_calibration_error(y_test, y_proba, n_bins))
print(expected_calibration_error(y_test, y_proba_cal, n_bins))

In [ ]:
figure, axis = plt.subplots(figsize=(8, 8))
disp = CalibrationDisplay.from_predictions(
    y_test,
    y_proba,
    n_bins=n_bins,
    name="perp",
    ax=axis,
    ref_line=True,
)
disp_cal = CalibrationDisplay.from_predictions(
    y_test,
    y_proba_cal,
    n_bins=n_bins,
    name="perp_cal",
    ax=axis,
    ref_line=True,
)
plt.grid(True)
plt.show()

In [ ]:
ece, prob_true, prob_pred, weights = expected_calibration_error(y_test_cv, y_proba_cv[:,1], n_bins)

print(f"ECE: {ece}")

In [ ]:
x = np.array([-1, 0.2, 6.4, 3.0, 1.6])
bins = np.array([0.0, 1.0, 2.5, 4.0, 10.0])
inds = np.digitize(x, bins)
inds

In [ ]:
bin_indices_train = np.digitize(y_proba_train_perp[:,1], bin_edges[1:-1])
print(bin_indices_train)
print(min(bin_indices_train))
print(max(bin_indices_train))

In [ ]:
proba_true_train = np.mean(y_train[bin_indices_train==0])
print(len(y_train[bin_indices_train==0]))
print(proba_true_train)

In [ ]:
proba_pred_train = np.mean(y_proba_train_perp[:,1][bin_indices_train==0])
print(len(y_proba_train_perp[:,1][bin_indices_train==0]))
print(proba_pred_train)

In [ ]:
pred_weights = get_weights(get_leaf_nodes(perp), perp.predict_nodes(X_test))

In [ ]:
bin_indices_test = np.digitize(y_proba_test_perp[:,1], bin_edges[1:-1])
print(bin_indices_test)
print(min(bin_indices_test))
print(max(bin_indices_test))

In [ ]:
proba_true_test = np.mean(y_test[bin_indices_test==0])
print(len(y_test[bin_indices_test==0]))
print(proba_true_test)

In [ ]:
proba_pred_test = np.mean(y_proba_test_perp[:,1][bin_indices_test==0])
print(len(y_proba_test_perp[:,1][bin_indices_test==0]))
print(proba_pred_test)

In [ ]:
pred_weights = get_weights(get_leaf_nodes(perp), pred_nodes_test)
pred_lower = np.sum(np.min(pred_weights, axis=2), axis=0) + perp.base_score
pred_lower = 1.0 / (1.0 + np.exp(-pred_lower))
pred_lower.shape

In [ ]:
print(np.mean(pred_lower[bin_indices_test==0]))

In [ ]:
pred_lower

In [ ]:
pred_weights = get_weights(get_leaf_nodes(perp), pred_nodes_test)
pred_upper = np.sum(np.max(pred_weights, axis=2), axis=0) + perp.base_score
pred_upper = 1.0 / (1.0 + np.exp(-pred_upper))
pred_upper.shape

In [ ]:
pred_upper

In [ ]:
import seaborn as sns

sns.displot(pred_upper - pred_lower)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(pred_lower, pred_upper, alpha=0.1)

In [ ]:
max(pred_upper - pred_lower)

In [ ]:
indices = np.random.randint(low=0, high=5, size=(pred_weights.shape[0], pred_weights.shape[1], 100))
new_pred_weights = np.take_along_axis(pred_weights, indices, axis=2)
print(f"New array shape: {new_pred_weights.shape}")

In [ ]:
new_pred_weights_sum = np.sum(new_pred_weights, axis=0) + perp.base_score
new_pred_weights_sum.shape

In [ ]:
sns.displot(new_pred_weights_sum[0])

In [ ]:
new_pred_weights_sum_proba = 1.0 / (1.0 + np.exp(-new_pred_weights_sum))
new_pred_weights_sum_proba.shape

In [ ]:
sns.displot(new_pred_weights_sum_proba[11000])

In [ ]:
sns.displot(np.max(new_pred_weights_sum_proba, axis=1)-np.min(new_pred_weights_sum_proba, axis=1))